# **`Post-Closeup Checkout of Rover`**

**Default Settings:**
\
`wired_seq_num = 0x01`
\
`itr=1`
\
`send_data_packet_to_wd_before_sniffing = False`
\
`specific_cmd_name_override = None`

**To sniff Serial from SBC without sending a command:**
1. `send_data_packet_to_wd_before_sniffing = False`

**To enter SERVICE mode:**
1. `itr = 1`

**To enter MISSION mode:**
1. `itr = 2`

**System Soft Reset:**  *(All systems except the WatchDog powered off)*
1. Enter **`SERVICE`** mode twice

**To Reset Specific Systems:**
1. specific_cmd_name_override = 'power-off-[system]`   
    *System can be one of the following:*
    - `herc`
    - `radio`
    - `motors`
    - `cameras`

**No telem from Herc after 200 sec**
1. `specific_cmd_name_override = 'power-off-herc'`
2. `specific_cmd_name_override = 'power-off-radio'`
3. `specific_cmd_name_override = 'power-on-radio'`
4. `specific_cmd_name_override = 'power-on-herc'`

**Misc Commands:**


## Additional Support:
See ***Pre-prepared commands list in `__command_aliases.py`***.

Logs of everything printed to the console will stored in `./raw-console-logs/` as `*.clog` test files under the name used for `settings['SAVE_FILE_PREFIX']`.

If you want to explore the Data Standards to build new commands, run:

`pyenv exec python datastandards_lookup.py`.

In [1]:
## SETUP:

from trans_tools import *
from __command_aliases import get_command
from IrisBackendv3.utils.basic import bytearray_to_spaced_hex as hexstr

seq_num = 0x01
blimp_cmds = ['setup', 'power-on', 'SetChargerEn', 'SetChargerPowerConnection', 'SetBatteryConnection', 'SetBatteryControlEnable', 'SetBatteryLatch']
cmd = blimp_cmds[0]
blimp_inputs = [False, True, 'OFF', 'ON', 'FORCE_HIGH', 'FORCE_LOW', 'LATCH_BATT_EN_PULSE_HIGH']
param = blimp_inputs[1]

send_data_packet_to_wd_before_sniffing = True
specific_cmd_name_override = cmd

## CE Test One - All Internal Rover Systems Pre-Deployment Enabled
# specific_cmd_name_override = 'setup' # ~8         32
# specific_cmd_name_override = 'power-on' # ~14.5   38

specific_cmd_name_override = '3v3-on'

# specific_cmd_name_override = 'power-off-herc' #    
# specific_cmd_name_override = 'power-off-radio' # 
# specific_cmd_name_override = 'power-on-herc' #    44
# specific_cmd_name_override = 'power-on-radio' #   45

# specific_cmd_name_override = 'reset-radio'
# specific_cmd_name_override = 'reset-herc' #         59 -> 63
## FAILED

# specific_cmd_name_override = 'SetBatteryConnection'
# param = False

print(specific_cmd_name_override,'  :  ',param)

3v3-on   :   True


In [2]:
## SETTINGS:

serial_device = '/dev/ttyUSB0'
ip="192.168.1.2"
port=8080

settings['SAVE_FILE_PREFIX'] = 'iris__post_vibe_11052021' # this is the prefix on all log files. make it something unique.
load_cache()
setup_logger(settings['SAVE_FILE_PREFIX']) # logs of stuff printed to console will be stored to `raw-console-logs`

# IF ERROR - update last number in FILE_PREFIX

In [3]:
step = specific_cmd_name_override
pathway, magic, command_name, kwargs, telem_pathway = get_command(step, param)
print(pathway, magic, command_name, kwargs)

packet = build_command_packet(seq_num, pathway, magic, command_name, kwargs)
packet_bytes = packet.encode()
print(hexstr(packet_bytes))

DataPathway.WIRED Magic.WATCHDOG_COMMAND WatchDogInterface_ResetSpecific {'reset_value': 'EN_3_3_POWER_ON'}
01 07 00 28 ee ff 00 c0 00 10 12


In [4]:
## Send Command:
connect_serial(device = serial_device)
if send_data_packet_to_wd_before_sniffing:
    send_packet(packet, pathway, ip, port)

Connection Success!


In [5]:
## Stream Telemetry:
stream_data_ip_udp_serial()
# Check for wireless telemetry in Wireshark. For parsing help, run: `pyenv exec python parse_pcap.py --help`

2021-11-05 14:57:09: INFO     > Command[#0] -> NO_ERROR[0x0]
2021-11-05 14:57:14: INFO     RS_SERVICE:	[Heat: OFF, Ctrl: ON] 	299.3°K -> 273.6°K +- 0.92K° 	Kp = 500 @ Duty Cycle: 0/65535
2021-11-05 14:57:21: INFO     RS_SERVICE:	[Heat: OFF, Ctrl: ON] 	299.3°K -> 273.6°K +- 0.92K° 	Kp = 500 @ Duty Cycle: 0/65535
2021-11-05 14:57:27: INFO     RS_SERVICE:	[Heat: OFF, Ctrl: ON] 	299.3°K -> 273.6°K +- 0.92K° 	Kp = 500 @ Duty Cycle: 0/65535
2021-11-05 14:57:34: INFO     RS_SERVICE:	[Heat: OFF, Ctrl: ON] 	299.3°K -> 273.6°K +- 0.92K° 	Kp = 500 @ Duty Cycle: 0/65535
2021-11-05 14:57:40: INFO     RS_SERVICE:	[Heat: OFF, Ctrl: ON] 	299.3°K -> 273.6°K +- 0.92K° 	Kp = 500 @ Duty Cycle: 0/65535
2021-11-05 14:57:47: INFO     RS_SERVICE:	[Heat: OFF, Ctrl: ON] 	299.3°K -> 273.6°K +- 0.92K° 	Kp = 500 @ Duty Cycle: 0/65535
